# Using fabric to setup a Cluster of Raspberry Pis

## 1. Installing and importing the libraries

I use two libraries: [Fabric](https://docs.fabfile.org/en/stable/) to manage the SSH connections and [Dotenv](https://pypi.org/project/python-dotenv/) to manage my SSH password and username without having to write down credentials it in the notebook, which is bad practice. I am using Python 3.11.10 .

To install Fabric:

`pip install fabric`

To install dotenv:

`pip install python-dotenv`

dotenv works by fetching environment variables from a file named `.env`  in the same directory as the script/notebook. I have mine setup with the three following lines:

SSH_PASSWORD=my_ssh_password

SSH_USER=my_ssh_username

ENV_TEST=hello

I can then import the libraries and load the .env file to retrieve my username and password.

In [1]:
import os
from fabric import *
from dotenv import load_dotenv

In [2]:
load_dotenv()
test = os.getenv("ENV_TEST") #to make sure that it is properly loaded
print(test)
ssh_password = os.getenv('SSH_PASSWORD')
ssh_user = os.getenv('SSH_USER')

hello


## Fabric Serial and Parallel functions

I have setup two function which allows me to send commands over ssh to my cluster either serially (one after another) or in parallel (all at the same time). Both of them require a dictionary of IP adresses to function.


In [ ]:
# This is a dictionary of my nodes' local IP adresses
ip_dict = {
    '192.168.8.128': 'node 1',
    '192.168.8.130': 'node 2',
    '192.168.8.129': 'node 3',
    '192.168.8.131': 'node 4',
    '192.168.8.127': 'node 5',
    '192.168.8.126': 'node 6',
    '192.168.8.140': 'node 7'
}

In [12]:
def run_command_serial(command, ip_dict = ip_dict, ssh_password = ssh_password, ssh_user = ssh_user):
    '''
    This function takes a shell command (command),
    a dictionary of IP adresses (ip_dict),
    as well as your ssh credential (ssh_password)(ssh_user).
    It then sends the command to each IP adress via ssh one by one,
    and prints out the output from the command.
    '''
    hosts = ip_dict.keys()
    
    group = SerialGroup(*hosts, user=ssh_user, connect_kwargs={"password": ssh_password})
    
    # Execute the command in parallel on all hosts
    results = group.run(command, hide=True)

    # Process results
    for connection, result in results.items():
        print(f"Host: {ip_dict[connection.host]}")
        print(f"Output: {result.stdout.strip()}")
        print(f"Error: {result.stderr.strip()}")
        print(f"Exit Status: {result.exited}")
        print("-" * 40)      
        
    return 0

def run_command_parallel(command, ip_dict = ip_dict, ssh_password = ssh_password, ssh_user = ssh_user):
    '''
    This function takes a shell command (command),
    a dictionary of IP adresses (ip_dict),
    as well as your ssh credential (ssh_password)(ssh_user).
    It then sends the command to all the IP address at the same time,
    and prints out the output from the command.
    Note that the output order will be randomly shuffled.
    '''
    
    hosts = ip_dict.keys()
    
    group = ThreadingGroup(*hosts, user=ssh_user, connect_kwargs={"password": ssh_password})
    
    # Execute the command in parallel on all hosts
    results = group.run(command, hide=True)

    # Process results
    for connection, result in results.items():
        print(f"Host: {ip_dict[connection.host]}")
        print(f"Output: {result.stdout.strip()}")
        print(f"Error: {result.stderr.strip()}")
        print(f"Exit Status: {result.exited}")
        print("-" * 40)      
        
    return 0


Testing that the functions are working properly:

In [10]:
print('___ Testing the run_command_serial() function: ___')
run_command_serial(command = 'echo "serial"; sleep 1; date')
print('___ Testing the run_command_parallel() function: ___')
run_command_parallel(command = 'echo "parallel"; sleep 1; date')

___ Testing the run_command_serial() function: ___
Host: node 1
Output: serial
Tue  5 Nov 15:31:27 CET 2024
Error: 
Exit Status: 0
----------------------------------------
Host: node 2
Output: serial
Tue  5 Nov 15:31:28 CET 2024
Error: 
Exit Status: 0
----------------------------------------
Host: node 3
Output: serial
Tue  5 Nov 15:31:29 CET 2024
Error: 
Exit Status: 0
----------------------------------------
Host: node 4
Output: serial
Tue  5 Nov 15:31:31 CET 2024
Error: 
Exit Status: 0
----------------------------------------
Host: node 5
Output: serial
Tue  5 Nov 15:31:32 CET 2024
Error: 
Exit Status: 0
----------------------------------------
Host: node 6
Output: serial
Tue  5 Nov 15:31:33 CET 2024
Error: 
Exit Status: 0
----------------------------------------
Host: node 7
Output: serial
Tue  5 Nov 15:31:35 CET 2024
Error: 
Exit Status: 0
----------------------------------------
___ Testing the run_command_parallel() function: ___
Host: node 6
Output: parallel
Tue  5 Nov 15:31:36

0

Notice that the timestamps for the 'serial' prints are all seperated by 1 second, while they are all the same for the 'parallel' prints. This means that the functions are working as intended.

## Docker Installation

Now that we can communicate with our cluster, we can start installing Docker. But first, let's make sure that our package installer is up to date:

In [ ]:
run_command_parallel('sudo apt-get update') # rather quick response
run_command_parallel('sudo apt-get upgrade -y') # much longer response time

### Installing Java (optional):

In [ ]:
#Installing Java
run_command_parallel("sudo apt install -y default-jre")
#Checking proper inntallation
run_command_parallel("java -version")

### Installing Docker:

In [11]:
run_command_parallel(command = 'curl -sSL https://get.docker.com | sh')

Host: node 8
Output: # Executing docker install script, commit: 6d51e2cd8c04b38e1c2237820245f4fc262aca6c
Client: Docker Engine - Community
 Version:           27.3.1
 API version:       1.47
 Go version:        go1.22.7
 Git commit:        ce12230
 Built:             Fri Sep 20 11:41:19 2024
 OS/Arch:           linux/arm64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          27.3.1
  API version:      1.47 (minimum version 1.24)
  Go version:       go1.22.7
  Git commit:       41ca978
  Built:            Fri Sep 20 11:41:19 2024
  OS/Arch:          linux/arm64
  Experimental:     false
 containerd:
  Version:          1.7.22
  GitCommit:        7f7fdf5fed64eb6a7caf99b3e12efcf9d60e311c
 runc:
  Version:          1.1.14
  GitCommit:        v1.1.14-0-g2c9f560
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-

0

Checking that Docker has been properly installed:

In [13]:
run_command_parallel(command = 'docker --version')

Host: node 5
Output: Docker version 27.3.1, build ce12230
Error: 
Exit Status: 0
----------------------------------------
Host: node 7
Output: Docker version 27.3.1, build ce12230
Error: 
Exit Status: 0
----------------------------------------
Host: node 4
Output: Docker version 27.3.1, build ce12230
Error: 
Exit Status: 0
----------------------------------------
Host: node 6
Output: Docker version 27.3.1, build ce12230
Error: 
Exit Status: 0
----------------------------------------
Host: node 3
Output: Docker version 27.3.1, build ce12230
Error: 
Exit Status: 0
----------------------------------------
Host: node 1
Output: Docker version 27.3.1, build ce12230
Error: 
Exit Status: 0
----------------------------------------
Host: node 2
Output: Docker version 27.3.1, build ce12230
Error: 
Exit Status: 0
----------------------------------------


0

Enable Docker to start on boot:

In [13]:
run_command_parallel(command = 'sudo systemctl enable docker')
run_command_parallel(command = 'sudo systemctl start docker')

Host: node 7
Output: 
Error: Synchronizing state of docker.service with SysV service script with /lib/systemd/systemd-sysv-install.
Executing: /lib/systemd/systemd-sysv-install enable docker
Exit Status: 0
----------------------------------------
Host: node 6
Output: 
Error: Synchronizing state of docker.service with SysV service script with /lib/systemd/systemd-sysv-install.
Executing: /lib/systemd/systemd-sysv-install enable docker
Exit Status: 0
----------------------------------------
Host: node 8
Output: 
Error: Synchronizing state of docker.service with SysV service script with /lib/systemd/systemd-sysv-install.
Executing: /lib/systemd/systemd-sysv-install enable docker
Exit Status: 0
----------------------------------------
Host: node 5
Output: 
Error: Synchronizing state of docker.service with SysV service script with /lib/systemd/systemd-sysv-install.
Executing: /lib/systemd/systemd-sysv-install enable docker
Exit Status: 0
----------------------------------------
Host: node 4

0

### Setting up Docker swarm

Initialize the Docker Swarm on the manager node:

In [ ]:
ip_manager = {'192.168.8.128': 'node 1'}
run_command_parallel(command='sudo docker swarm init --advertise-addr <MANAGER_NODE_IP>', 
                     ip_dict=ip_manager,
                     ssh_password=ssh_password,
                     ssh_user=ssh_user)

After initializing the swarm, Docker will output a command with a token that you should run on each worker node.

Adding worker nodes to the Swarm:

In [14]:
ip_workers = {
    '192.168.8.130': 'node 2',
    '192.168.8.129': 'node 3',
    '192.168.8.131': 'node 4',
    '192.168.8.127': 'node 5',
    '192.168.8.126': 'node 6',
    '192.168.8.140': 'node 7'
}
run_command_parallel(command = 'sudo docker swarm join --token <token> <MANAGER_NODE_IP>:2377',
                     ip_dict=ip_workers,
                     ssh_password=ssh_password,
                     ssh_user=ssh_user)

Host: node 7
Output: This node joined a swarm as a worker.
Error: 
Exit Status: 0
----------------------------------------
Host: node 8
Output: This node joined a swarm as a worker.
Error: 
Exit Status: 0
----------------------------------------
Host: node 5
Output: This node joined a swarm as a worker.
Error: 
Exit Status: 0
----------------------------------------
Host: node 6
Output: This node joined a swarm as a worker.
Error: 
Exit Status: 0
----------------------------------------
Host: node 4
Output: This node joined a swarm as a worker.
Error: 
Exit Status: 0
----------------------------------------
Host: node 3
Output: This node joined a swarm as a worker.
Error: 
Exit Status: 0
----------------------------------------


0

Verifying the Swarm cluster on the manager node:

In [ ]:
run_command_parallel(command='sudo docker node ls', 
                     ip_dict=ip_manager,
                     ssh_password=ssh_password,
                     ssh_user=ssh_user)

## Setting up Portainer

### Installing Portainer on the Docker manager Node

Portainer is a lightweight, easy-to-use UI for managing Docker environments, including Docker Swarm clusters. It has a web interface that allows you to manage and monitor containers, images, volumes, networks, and more. Portainer will be running on the manager node’s IP address.

In [ ]:
run_command_parallel(command='sudo docker volume create portainer_data', 
                     ip_dict=ip_manager,
                     ssh_password=ssh_password,
                     ssh_user=ssh_user)

In [ ]:
run_command_parallel(command='sudo docker run -d -p 9000:9000 --name=portainer --restart=always -v /var/run/docker.sock:/var/run/docker.sock -v portainer_data:/data portainer/portainer-ce', 
                     ip_dict=ip_manager,
                     ssh_password=ssh_password,
                     ssh_user=ssh_user)

### Accessing the Portainer UI

Accessing the Portainer UI is really easy and can be done from any device on the network by simply opening a browser and going to `http://<MANAGER_NODE_IP>:9000`. You'll be prompted to create an admin user the first time you access it.

## Conclusion

That's it ! The cluster is now setup and can be used to run and monitor docker containers using the Portainer User Interface.